In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris, load_digits
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, OneHotEncoder

class MatrixMultiplicationLayer:
    def __init__(self, X, W):
        self.X = X
        self.W = W
        
    def forward(self):
        self.output = np.dot(self.X, self.W)

    def backward(self):
        self.doutput_dX = (self.W)
        self.doutput_dW = (self.X).T 

class BiasAdditionLayer:
    
    def __init__(self, output : np.ndarray , B : np.ndarray):
        self.B = B
        self.output = output
    
    def forward(self):
        self.output = self.output + self.B

    def backward(self):
        self.doutput_dB = np.identity(self.B.shape[1])

class MeanSquaredLossLayer:

    def __init__(self, target: np.ndarray, predicted: np.ndarray):

        self.predicted = predicted
        self.target = target 
        
    def forward(self):
        error = self.predicted - self.target
        self.L = np.mean(np.square(error))

    def backward(self):
        self.dL_dpredicted = (2 / len(self.target)) * (self.predicted - self.target).T
        
# softmax(x)i = e^(x_i)/∑ j={1,n}e^(x_j)
# Here,softmax(x)i represents the i-th element of the output vector after applying the softmax function to 
# x, and x_i represents the i-th element of the input vector x. The softmax function normalizes the values of 
# x into a probability distribution where each element of the output vector 
# softmax(x) is in the range [0, 1] and the sum of all elements equals 1.

class SoftmaxLayer:

    def __init__(self, Z):
        self.Z = Z 
        
    def forward(self):
        exp_X = np.exp(self.Z - np.max(self.Z, axis=1, keepdims=True))
        self.output = exp_X / np.sum(exp_X, axis=1, keepdims=True)

    def backward(self):
        self.doutput_dZ = np.diag(self.output.reshape(-1)) - (self.output.T)@((self.output))
        
# sigmoid(x)= 1/1+e^−x
# Here,sigmoid(x) represents the output of the sigmoid function for input x. The sigmoid function maps any 
# real-valued number to the range [0, 1]. It has an S-shaped curve, with the function outputting values 
# close to 0 for large negative inputs and values close to 1 for large positive inputs. It is often used 
# to convert raw scores into probabilities in binary classification tasks or as an activation function 
# in neural networks

class SigmoidLayer:

    def __init__(self, Z):
        self.Z = Z
        
    def forward(self):
        self.output = 1 / (1 + np.exp(-self.Z))

    def backward(self):
        self.doutput_dZ = np.diag(np.multiply(self.output,(1 - self.output)).reshape(-1))
        
# CrossEntropyLoss= (1/N) * ∑ i={1,N} ∑ c={1,C} (yi,c * log(p i,c)) 
# Here:

# N is the number of samples.

# C is the number of classes.

# yi,c is a binary indicator (0 or 1) for whether class c is the correct classification for sample i.

# pi,c is the predicted probability that sample i belongs to class c.

# The Cross-Entropy Loss penalizes models more heavily when they make large errors in classification, 
# as the logarithm term amplifies the loss for confident, incorrect predictions. It is commonly used as 
# the loss function in multi-class classification problems, especially when combined with softmax activation 
# in the output layer of neural networks.

class CrossEntropyLossLayer:

    def __init__(self, target,predicted):
        self.target = target
        self.predicted = predicted
        
    def forward(self):
        epsilon = 1e-15
        self.L = -np.sum(self.target * np.log(self.predicted + epsilon))
    def backward(self):
        self.dL_dpredicted =  -1*((self.target) / (len(self.predicted)+1e-40)).T

# LinearActivation(x)=x
# While Linear Activation is rarely used within hidden layers of neural networks due to its inability 
# to introduce non-linearity, it is often used as the activation function in the output layer of regression 
# models, where the network is tasked with predicting continuous values.
class LinearActivation:

    def __init__(self, Z):
        self.Z = Z 
        
    def forward(self):
        self.output = self.Z
         
    def backward(self):
        self.doutput_dZ = np.identity(self.Z.shape[1])
        
# TanhActivation(x)= (e^x - e^−x) /(e^x + e^−x)
# The Tanh Activation function is often used in hidden layers of neural networks to introduce non-linearity. 
# It is particularly useful when dealing with data that has negative values since it can map negative inputs to 
# negative outputs.

class TanhActivation:

    def __init__(self, Z):
        self.Z = Z
        
    def forward(self):
        self.output = np.tanh(self.Z)
    
    def backward(self):
        self.doutput_dZ =  np.diag(1 - self.output.reshape(-1)**2)
# ReLU(x)=max(0,x)
# ReLU has become the default choice for many neural network architectures due to its simplicity and 
# effectiveness in combating the vanishing gradient problem during training. It is particularly effective in 
# deep neural networks, where it helps mitigate the vanishing gradient problem by allowing gradients to flow 
# more freely during backpropagation.

class ReLUActivation:

    def __init__(self, Z): 
        self.Z = Z 
        self.Leak = 0.01
        
    def forward(self):
        self.output = np.maximum(0,self.Z)
      
    def backward(self):
        self.doutput_dZ = np.diag([1. if x>=0 else 0.01 for x in self.output.reshape(-1)])

In [2]:
from sklearn.model_selection import train_test_split

def load_data_sklearn(dataset_name='california', normalize_X=False, normalize_y=False, one_hot_encode_y=False, test_size=0.2):
    if dataset_name == 'california':
        california_data = fetch_california_housing()
        data = {'data': california_data.data, 'target': california_data.target}       
    elif dataset_name == 'iris':
        iris_data = load_iris()
        data = {'data': iris_data.data, 'target': iris_data.target}
    elif dataset_name == 'mnist':
        mnist_data = load_digits()
        data = {'data': (mnist_data.data >= 8), 'target': (mnist_data.target).astype(int)}

    X = data['data']
    y = data['target'].reshape(-1, 1)

    if normalize_X:
        normalizer = Normalizer()
        X = normalizer.fit_transform(X)

    if normalize_y:
        normalizer = Normalizer()
        y = normalizer.fit_transform(y)

    if one_hot_encode_y:
        encoder = OneHotEncoder()
        y = encoder.fit_transform(y).toarray()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    return X_train, y_train, X_test, y_test

In [3]:
class Layer:
    def __init__(self, dimenshion_of_input,output_neuron, activation_name="linear", seed=42):
        np.random.seed(seed)
        
        self.dimenshion_of_input = dimenshion_of_input
        self.output_neuron = output_neuron
        
        # random initialization of input X  and output Z
        self.X = np.random.random((1, self.dimenshion_of_input))   # assigned during SGD
        self.Z = np.random.random((1, self.output_neuron))
        
        self.W = np.random.random((dimenshion_of_input, output_neuron)) * \
            np.sqrt(2 / (dimenshion_of_input + output_neuron))
        self.B = np.random.random((1, output_neuron))*np.sqrt(2 / (1 + output_neuron))
        
        self.multiply_layer = MatrixMultiplicationLayer(self.X, self.W)
        self.bias_layer = BiasAdditionLayer(self.B, self.B)

        if activation_name == 'linear':
            self.activation_layer = LinearActivation(self.Z)
        elif activation_name == 'sigmoid':
            self.activation_layer = SigmoidLayer(self.Z)
        elif activation_name == 'softmax':
            self.activation_layer = SoftmaxLayer(self.Z)
        elif activation_name == 'tanh':
            self.activation_layer = TanhActivation(self.Z)
        elif activation_name == 'relu':
            self.activation_layer = ReLUActivation(self.Z)
            
    def forward_layer(self):
        self.multiply_layer.forward()
        self.bias_layer.output = self.multiply_layer.output
        self.bias_layer.forward()
        self.activation_layer.Z = self.bias_layer.output
        self.activation_layer.forward()
        self.Z = self.activation_layer.output
    def backward_layer(self):
        self.activation_layer.backward()
        self.bias_layer.backward()
        self.multiply_layer.backward()

class NeuralNetwork(Layer):
    """
    Input  - layers : list of layer objects , loss_name : Name of loss layer
    """

    # [ "mean_squared", "cross_entropy"]
    def __init__(self, layers, loss_name="mean_squared", learning_rate=0.01, seed=42):
        np.random.seed(seed)

        self.layers = layers
        self.n_layers = len(layers)  # number of layers in neural network
        self.learning_rate = learning_rate

        self.inp_shape = self.layers[0].X.shape
        self.out_shape = self.layers[-1].Z.shape

        # random initialization of input X  and output Z
        self.X = np.random.random(self.inp_shape)   # assigned during SGD
        self.Y = np.random.random(self.out_shape)  # output of neural network

        # define loss layer
        if loss_name == "mean_squared":
            self.loss_layer = MeanSquaredLossLayer(self.Y, self.Y)
        if loss_name == "cross_entropy":
            self.loss_layer = CrossEntropyLossLayer(self.Y, self.Y)

    def forward(self):
        self.layers[0].X = self.X
        self.loss_layer.target = self.Y

        self.layers[0].forward_layer()
        for i in range(1, self.n_layers):
            self.layers[i].X = self.layers[i-1].Z
            self.layers[i].forward_layer()

        self.loss_layer.predicted = self.layers[-1].Z
        self.loss_layer.forward()

    def backward(self):

#         self.loss_layer.Z = self.Y
        self.loss_layer.backward()
        self.grad_nn = self.loss_layer.dL_dpredicted
        for i in range(self.n_layers-1, -1, -1):
            self.layers[i].backward_layer()

            dL_dZ = np.dot(
                self.layers[i].activation_layer.doutput_dZ, self.grad_nn)
            dL_dW = np.dot(self.layers[i].multiply_layer.doutput_dW, dL_dZ.T)
            dL_dB = np.dot(self.layers[i].bias_layer.doutput_dB, dL_dZ).T

            # Update W & B
            self.layers[i].W -= self.learning_rate*dL_dW
            self.layers[i].B -= self.learning_rate*dL_dB

            # Update outer_grad
            self.grad_nn = np.dot(self.layers[i].multiply_layer.doutput_dX, dL_dZ)

            del dL_dZ, dL_dW, dL_dB

In [4]:
def createLayers(inp_shape, layers_sizes, layers_activations):
    layers = []
    n_layers = len(layers_sizes)
    layer_0 = Layer(inp_shape, layers_sizes[0], layers_activations[0])
    layers.append(layer_0)
    inp_shape_next = layers_sizes[0]
    for i in range(1, n_layers):
        layer_i = Layer(inp_shape_next, layers_sizes[i], layers_activations[i])
        layers.append(layer_i)
        inp_shape_next = layers_sizes[i]

    out_shape = inp_shape_next
    return inp_shape, out_shape, layers

def SGD_NeuralNetwork(X_train,
                      y_train,
                      X_test,
                      y_test,
                      nn,
                      inp_shape=1,   # dimension of input
                      out_shape=1,   # dimension of output
                      n_iterations=1000,
                      task="regression"  # [ "regression", "classification"]
                      ):
    # Define a custom progress bar function
    def progress_bar(iteration, total_iterations):
        progress = (iteration + 1) / total_iterations
        bar_length = 30
        filled_length = int(bar_length * progress)
        bar = '=' * filled_length + '-' * (bar_length - filled_length)
        print(f'\rTraining ... [{bar}] {progress * 100:.2f}%', end='', flush=True)
        if iteration == total_iterations - 1:
            print()

    # Initialize iteration counter
    iteration = 0

    # Loop until iteration reaches n_iterations
    while iteration < n_iterations:
        # Display the progress bar
        progress_bar(iteration, n_iterations)

        random_index = np.random.randint(len(X_train))
        X_sample = X_train[random_index, :].reshape(1, inp_shape)
        Y_sample = y_train[random_index, :].reshape(1, out_shape)

        nn.X = X_sample
        nn.Y = Y_sample

        nn.forward()  # Forward Pass
        nn.backward()  # Backward Pass

        # Increment the iteration counter
        iteration += 1

    # Clear the progress bar after training
    print('\033[K', end='', flush=True)

    # Lets run ONLY forward pass for train and test data and check accuracy/error

    if task == "regression":
        nn.X = X_train
        nn.Y = y_train
        nn.forward()
        train_error = nn.loss_layer.L
        nn.X = X_test
        nn.Y = y_test

        nn.forward()

        test_error = nn.loss_layer.L

        if isinstance(nn.loss_layer, MeanSquaredLossLayer):
            print("Mean Squared Loss Error (Train Data)  : %0.5f" % train_error)
            print("Mean Squared Loss Error (Test Data)  : %0.5f" % test_error)

    if task == "classification":
        nn.X = X_train
        nn.Y = y_train
        nn.forward()
        y_true = np.argmax(y_train, axis=1)
        y_pred = np.argmax(nn.loss_layer.predicted, axis=1)
        acc = 1 * (y_true == y_pred)
        print("Classification Accuracy (Training Data ): {0}/{1} = {2} %".format(
            sum(acc), len(acc), sum(acc) * 100 / len(acc)))

        nn.X = X_test
        nn.Y = y_test
        nn.forward()
        y_true = np.argmax(y_test, axis=1)
        y_pred = np.argmax(nn.loss_layer.predicted, axis=1)
        acc = 1 * (y_true == y_pred)
        print("Classification Accuracy (Testing Data ): {0}/{1} = {2} %".format(
            sum(acc), len(acc), sum(acc) * 100 / len(acc)))


# CALIFORNIA

In [5]:
X_train, y_train, X_test, y_test = load_data_sklearn('california', normalize_X=True, normalize_y=False, test_size=0.2)

# One output neural with Linear activation and least mean squared error

In [6]:
inp_shape = X_train.shape[1]
layers_sizes = [1]
layers_activations = ['linear']

inp_shape, out_shape, layers = createLayers(inp_shape, layers_sizes, layers_activations)
loss_nn = 'mean_squared'

nn = NeuralNetwork(layers, loss_nn, learning_rate=0.1)

SGD_NeuralNetwork(X_train,y_train,X_test,y_test,nn,inp_shape, out_shape,n_iterations=12111,task="regression")

Training ... [==============================] 100.00%
Mean Squared Loss Error (Train Data)  : 2.03453
Mean Squared Loss Error (Test Data)  : 2.05931


# 2 layers. Layer 1 with 13 output neurons with sigmoid activation. Layer 2 with one output neuron and linear activation. use mean squared loss

In [7]:
inp_shape = X_train.shape[1]
layers_sizes = [13,1]
layers_activations = ['sigmoid','linear']

inp_shape, out_shape, layers = createLayers(inp_shape, layers_sizes, layers_activations)
loss_nn = 'mean_squared'

nn = NeuralNetwork(layers, loss_nn, learning_rate=0.01)

SGD_NeuralNetwork(X_train,y_train,X_test,y_test,nn,inp_shape, out_shape,n_iterations=1000,task="regression")

Training ... [==============================] 100.00%
Mean Squared Loss Error (Train Data)  : 1.38606
Mean Squared Loss Error (Test Data)  : 1.42777


# Three layers. Layer 1 with 13 output neurons with sigmoid activation. Layer 2 with 13 output neurons and sigmoid activation. Layer 3 with one output neuron and linear activation. use mean squared loss

In [8]:
inp_shape = X_train.shape[1]
layers_sizes = [13,13,1]
layers_activations = ['sigmoid','sigmoid','linear']

inp_shape, out_shape, layers = createLayers(inp_shape, layers_sizes, layers_activations)
loss_nn = 'mean_squared'

nn = NeuralNetwork(layers, loss_nn, learning_rate=0.001)

SGD_NeuralNetwork(X_train,y_train,X_test,y_test,nn,inp_shape, out_shape,n_iterations=1000,task="regression")

Training ... [==============================] 100.00%
Mean Squared Loss Error (Train Data)  : 1.32945
Mean Squared Loss Error (Test Data)  : 1.36843
